In [2]:
import pandas as pd
import numpy as np
import tejapi
import statsmodels.formula.api as smf
tejapi.ApiConfig.api_key = "maQNDP8Vb5bBB8b55kCkR2TC6s65Eg"
tejapi.ApiConfig.api_base="http://10.10.10.66"
tejapi.ApiConfig.ignoretz = True

In [3]:
df1 = tejapi.get('TWN/ACRQMTAB', #從TEJ api撈取所需要的資料
                chinese_column_name = True,
                paginate = True,
#                 mdate = {'gt':'2018-01-01'},
                opts={'columns':['coid','mdate','xcdt']})

In [4]:
df1

,公司,年/月,狀況
None,,,
0,1101,2008-03-01,
1,1101,2008-06-01,
2,1101,2008-09-01,
3,1101,2008-12-01,
4,1101,2009-03-01,
...,...,...,...
94188,9978,2020-12-01,上市前
94189,9978,2021-06-01,上市前
94190,9978,2021-12-01,上市前


先確認狀況有幾種類別

In [5]:
df1['狀況'].unique()

array(['', '全交下市', '上市前', '財務未滿三年', '成立未滿4年', '下市', '撤公開/結束', '-'],
      dtype=object)

將上市前股票排除

In [6]:
df2 = df1[df1['狀況'] != '上市前']

In [7]:
df2

,公司,年/月,狀況
None,,,
0,1101,2008-03-01,
1,1101,2008-06-01,
2,1101,2008-09-01,
3,1101,2008-12-01,
4,1101,2009-03-01,
...,...,...,...
94175,9965,2007-06-01,
94176,9965,2007-12-01,
94177,9965,2008-06-01,


將全交下市及下市 股票設為1 其餘為0

In [8]:
df2['狀況'] = df2[['狀況']].applymap(lambda x:1 if x =='全交下市' or x =='下市'  else 0)

/var/folders/gn/7b_mmd0s64q0kkr42rncl0jc0000gn/T/ipykernel_26068/1047614279.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['狀況'] = df2[['狀況']].applymap(lambda x:1 if x =='全交下市' or x =='下市'  else 0)


抓變數會用到的財報資料

In [12]:
a1 = tejapi.get('TWN/AIFIN', #從TEJ api撈取所需要的資料
                chinese_column_name = True,
                paginate = True,
                mdate = {'gt':'2008-01-01', 'lt':'2011-01-01'},
                acc_code = ['R678', '0010','2341','2402','MV','1000', 'R607','R11V', 'R505'])

In [14]:
a2 = tejapi.get('TWN/AIFIN', #從TEJ api撈取所需要的資料
                chinese_column_name = True,
                paginate = True,
                mdate = {'gt':'2011-01-01', 'lt':'2014-01-01'},
                acc_code = ['R678', '0010','2341','2402','MV','1000', 'R607','R11V', 'R505'])

In [25]:
a3 = tejapi.get('TWN/AIFIN', #從TEJ api撈取所需要的資料
                chinese_column_name = True,
                paginate = True,
                mdate = {'gt':'2014-01-01', 'lt':'2017-01-01'},
                acc_code = ['R678', '0010','2341','2402','MV','1000', 'R607','R11V', 'R505'])

In [26]:
a4 = tejapi.get('TWN/AIFIN', #從TEJ api撈取所需要的資料
                chinese_column_name = True,
                paginate = True,
                mdate = {'gt':'2017-01-01', 'lt':'2020-01-01'},
                acc_code = ['R678', '0010','2341','2402','MV','1000', 'R607','R11V', 'R505'])

In [27]:
a5 = tejapi.get('TWN/AIFIN', #從TEJ api撈取所需要的資料
                chinese_column_name = True,
                paginate = True,
                mdate = {'gt':'2020-01-01'},
                acc_code = ['R678', '0010','2341','2402','MV','1000', 'R607','R11V', 'R505'])

In [37]:
acc = pd.concat([a1,a2,a3,a4,a5])

In [39]:
acc1 = acc.pivot_table(values='數值', index=['公司','年/月'], columns='會計科目') #用pivot table將會計科目放到columns上

In [40]:
acc1

會計科目                   0010      1000      2341     2402  MV  R11V   R505  \
公司     年/月                                                                  
000016 2008-06-01  853115.0  206761.0   96354.0  38116.0 NaN   NaN  24.24   
       2008-12-01  794817.0  136689.0  108128.0  18324.0 NaN   NaN  17.20   
       2009-06-01  789105.0  127192.0  113626.0  30467.0 NaN   NaN  16.12   
       2009-12-01  828313.0  158918.0  121108.0  11891.0 NaN   NaN  19.19   
       2010-06-01  723437.0  293459.0  129978.0  28978.0 NaN   NaN  40.56   
...                     ...       ...       ...      ...  ..   ...    ...   
Z5389  2021-03-01  671435.0  506622.0  -69187.0      NaN NaN  3.12  75.45   
       2021-06-01  666327.0  504462.0  -72135.0      NaN NaN  2.66  75.71   
       2021-09-01  741529.0  477884.0  -78355.0      NaN NaN  1.59  64.45   
       2021-12-01  669558.0  413791.0  -86233.0      NaN NaN  0.44  61.80   
       2022-03-01  791483.0  503928.0  -54445.0      NaN NaN  4.35  63.67   

會計科目               R607  R678  
公司     年/月                     
000016 2008-06-01  0.67   NaN  
       2008-12-01  0.63   NaN  
       2009-06-01  0.50   NaN  
       2009-12-01  0.52   NaN  
       2010-06-01  0.71   NaN  
...                 ...   ...  
Z5389  2021-03-01  0.38   NaN  
       2021-06-01  0.29   NaN  
       2021-09-01  0.24   NaN  
       2021-12-01  0.24   NaN  
       2022-03-01  0.46   NaN  

[125489 rows x 9 columns]

In [41]:
acc1['X1'] = (acc1['R678']/acc1['0010'])*100
acc1['X2'] = (acc1['2341']/acc1['0010'])*100
acc1['X3'] = (acc1['2402']/acc1['0010'])*100
acc1['X4'] = (acc1['MV']/acc1['1000'])*100
acc1 = acc1.rename(columns = {'R607':'X5', 'R11V':'X6', 'R505':'X7'})
acc2 = acc1[['X1','X2','X3','X4','X5','X6','X7']]

In [42]:
df3 = df2.merge(acc2, on=['公司','年/月']) #合併應變數跟自變數

In [43]:
df3 = df3.rename(columns = {'狀況':'Y'})

In [44]:
df3

,公司,年/月,Y,X1,X2,X3,X4,X5,X6,X7
0,1101,2008-03-01,0,14.380415,8.672234,1.388886,215.029263,0.38,NaN,46.86
1,1101,2008-06-01,0,6.049163,6.180613,2.820563,135.481445,0.40,NaN,51.40
2,1101,2008-09-01,0,3.295652,6.987726,4.139193,63.789175,0.43,NaN,52.06
3,1101,2008-12-01,0,2.454791,7.510703,4.874842,91.828908,0.41,NaN,51.70
4,1101,2009-03-01,0,5.075376,7.369621,0.578818,92.064356,0.38,NaN,51.81
...,...,...,...,...,...,...,...,...,...,...
89902,9962,2021-06-01,0,55.079065,5.972435,4.331177,503.227118,1.83,4.05,22.58
89903,9962,2021-09-01,0,43.650040,6.941514,6.155575,258.033600,1.64,6.41,39.97
89904,9962,2021-12-01,0,49.451508,10.478023,10.505071,270.381893,1.85,10.59,35.41
89905,9962,2022-03-01,0,57.969112,14.440720,3.158543,508.980860,2.10,2.48,24.53


linear probility model

In [45]:
result_ols = smf.ols('Y ~ X1 + X2 + X3 + X4 + X5 + X6 + X7', data=df3).fit()
print(result_ols.summary())

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     24.87
Date:                Mon, 08 Aug 2022   Prob (F-statistic):           3.80e-34
Time:                        15:28:03   Log-Likelihood:             1.2052e+05
No. Observations:               68070   AIC:                        -2.410e+05
Df Residuals:                   68062   BIC:                        -2.410e+05
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0017      0.001      2.543      0.0

probit model

In [72]:
result_probit = smf.probit('Y ~ X1 + X2 + X3 + X4 + X5 + X6 + X7 ', data=df3).fit()
print(result_probit.summary())

Optimization terminated successfully.
         Current function value: 0.012075
         Iterations 10
                          Probit Regression Results                           
Dep. Variable:                      Y   No. Observations:                68070
Model:                         Probit   Df Residuals:                    68062
Method:                           MLE   Df Model:                            7
Date:                Mon, 08 Aug 2022   Pseudo R-squ.:                 0.03904
Time:                        15:35:46   Log-Likelihood:                -821.98
converged:                       True   LL-Null:                       -855.37
Covariance Type:            nonrobust   LLR p-value:                 6.590e-12
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -2.9552      0.125    -23.598      0.000      -3.201      -2.710
X1             0.0030      0

logit model

In [71]:
result_Logit = smf.logit('Y ~ X1 + X2 + X3 + X4 + X5 + X6 + X7  ', data=df3).fit()
print(result_Logit.summary())

Optimization terminated successfully.
         Current function value: 0.012104
         Iterations 11
                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                68070
Model:                          Logit   Df Residuals:                    68062
Method:                           MLE   Df Model:                            7
Date:                Mon, 08 Aug 2022   Pseudo R-squ.:                 0.03677
Time:                        15:30:46   Log-Likelihood:                -823.92
converged:                       True   LL-Null:                       -855.37
Covariance Type:            nonrobust   LLR p-value:                 3.964e-11
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -6.3726      0.381    -16.710      0.000      -7.120      -5.625
X1             0.0078      0

台積電

In [50]:
TSMC = df3[df3['公司']=='2330']

In [51]:
a = TSMC.iloc[-1:,:] #取得最新一筆資料

In [52]:
result_ols.predict(a) #linear probility model

17443    0.000587
dtype: float64

In [53]:
result_probit.predict(a) #probit model

17443    0.001299
dtype: float64

In [54]:
result_Logit.predict(a) #logit model

17443    0.001398
dtype: float64

英瑞ky

In [55]:
Enterex = df3[df3['公司']=='1592']

In [56]:
b = Enterex.iloc[-1:,:]

In [57]:
result_ols.predict(b) #linear probility model

8610    0.002987
dtype: float64

In [58]:
result_probit.predict(b) #probit model

8610    0.002103
dtype: float64

In [59]:
result_Logit.predict(b) #logit model

8610    0.002126
dtype: float64

華映

In [60]:
CPT = df3[df3['公司']=='2475']

In [61]:
d = CPT.iloc[-1:,:]

In [62]:
result_ols.predict(d) #linear probility model

23362    0.007564
dtype: float64

In [63]:
result_probit.predict(d) #probit model

23362    0.003495
dtype: float64

In [64]:
result_Logit.predict(d) #logit model

23362    0.003349
dtype: float64

和進

In [94]:
hold = df3[df3['公司']=='3191']

In [95]:
e = hold.iloc[-1:,:]

In [96]:
result_ols.predict(e) #linear probility model

35206    0.004523
dtype: float64

In [97]:
result_probit.predict(e) #probit model

35206    0.002643
dtype: float64

In [98]:
result_Logit.predict(e) #logit model

35206    0.002426
dtype: float64